
# Step 1 — Random Walk Intuition (Brownian Motion warm‑up)

**Goal:** build a clear mental model of Brownian motion starting from a simple 1‑D random walk: a point that moves right or left by one small step at each tick of time, at random.

What you will do in this notebook:
- Simulate a basic random walk (±1 steps).
- See how the **spread grows with time**.
- Look at the **distribution at a fixed time**.
- Check that **successive steps are independent** (no memory).
- Connect this to the idea of Brownian motion as a “limit” of finer and finer steps.


In [ ]:

# Setup
import numpy as np
import matplotlib.pyplot as plt

def random_walk(steps=1000, seed=None):
    """Return positions for a 1-D random walk with ±1 steps.
    Shape: (steps+1,). Position starts at 0.
    """
    rng = np.random.default_rng(seed)
    # Draw ±1 with equal probability
    step_vals = rng.choice([-1, 1], size=steps)
    pos = np.empty(steps + 1, dtype=int)
    pos[0] = 0
    pos[1:] = np.cumsum(step_vals)
    return pos



## Experiment 1 — Visualize several random walks

**What to expect:** zigzags with no obvious trend, centered roughly around 0. As time grows, paths spread out.


In [ ]:

T = 1000  # number of time ticks
paths = 6 # how many walks to visualize

plt.figure()
for k in range(paths):
    pos = random_walk(T, seed=k)
    plt.plot(range(T+1), pos)
plt.xlabel("time tick")
plt.ylabel("position")
plt.title("Random walks (±1 steps)")
plt.show()



## Experiment 2 — Distribution at a fixed time

Pick a specific time (e.g., tick 500) and look at all the positions we get across many independent walks.

**Key idea:** At time *t*, positions are typically near 0, and extreme values are rarer. As *t* increases, the spread grows.


In [ ]:

T = 1000
tick = 500
num_paths = 10000

# Collect positions at the chosen tick across many walks
samples = np.empty(num_paths, dtype=int)
for k in range(num_paths):
    pos = random_walk(T, seed=k)
    samples[k] = pos[tick]

# Summary stats and histogram
print(f"Mean at tick {tick}: {samples.mean():.3f}")
print(f"Variance at tick {tick}: {samples.var(ddof=1):.3f}")

plt.figure()
plt.hist(samples, bins=51)
plt.xlabel(f"position at tick {tick}")
plt.ylabel("count")
plt.title("Distribution of positions at a fixed time")
plt.show()



## Experiment 3 — How spread grows with time

Compute the sample variance of positions across many walks at different times.  
**Claim:** the variance grows roughly **linearly** with time.


In [ ]:

T = 1000
num_paths = 5000

# Generate many walks once for efficiency
walks = np.empty((T+1, num_paths), dtype=int)
for k in range(num_paths):
    walks[:, k] = random_walk(T, seed=k)

# Estimate variance across paths for a grid of times
check_ticks = np.linspace(50, T, 10, dtype=int)  # avoid very small times at start
vars_est = []
for t in check_ticks:
    vars_est.append(walks[t].var(ddof=1))

plt.figure()
plt.plot(check_ticks, vars_est, marker="o")
plt.xlabel("time tick")
plt.ylabel("sample variance across paths")
plt.title("Variance grows approximately linearly with time")
plt.show()

# Quick and dirty linear fit (least squares) to show linear trend
A = np.vstack([check_ticks, np.ones_like(check_ticks)]).T
m, b = np.linalg.lstsq(A, np.array(vars_est), rcond=None)[0]
print(f"Linear fit: variance ≈ {m:.3f} * time + {b:.3f}")



## Experiment 4 — Independence of steps (no memory)

Build one long walk, extract its steps (±1), and compute the **lag-1 autocorrelation**.  
Under independence, this should be close to 0.


In [ ]:

# Build steps explicitly
rng = np.random.default_rng(123)
steps = rng.choice([-1, 1], size=100_000)
# lag-1 autocorrelation: corr(steps[:-1], steps[1:])
x = steps[:-1].astype(float)
y = steps[1:].astype(float)
corr = np.corrcoef(x, y)[0, 1]
print(f"Lag-1 autocorrelation ≈ {corr:.4f} (should be near 0)")



## Concept bridge — From random walk to Brownian motion

If you make the time steps **smaller** and the moves **smaller** (scaled like the square‑root of the time step), and you look at the curve in continuous time, the discrete random walk converges to **Brownian motion**.

- Brownian motion starts at 0.
- Its average position at any fixed time is 0.
- Its spread (variance) at time *t* grows proportionally to *t*.
- It has **independent** increments: what happens next does not depend on the past.

We will use this continuous‑time object as the “noise” building block in finance later.



## Try it yourself

1. Change the number of paths and ticks in *Experiment 1*. What happens to the envelope of the trajectories?
2. In *Experiment 2*, try different fixed times (100, 700, 1000). How do the mean and variance change?
3. In *Experiment 3*, increase `num_paths` to stabilize the variance curve. Does the fitted slope get closer to 1?
4. Replace ±1 steps by ±2 (or a mix like −2, −1, +1, +2). How does that change the spread?
5. Bonus: compute the **lag-5** autocorrelation of steps in *Experiment 4* and check it is also near 0.
